In [1]:
from datetime import datetime as dt
def main():
#---Select here the starting date
    year = 2019
    month = 9
    day = 29
#---Importing packagesfrom datetime import datetime as dt
    from datetime import timedelta
    import gzip
    import glob
    import os
    import shutil
    import sys
    import time
    import requests
    import pandas as pd
    import numpy as np
    from math import log,sqrt
    import gc

#---Importing raw data of trade and quote
    # https://public.bitmex.com/?prefix=data/trade/
    endpoint = 'https://s3-eu-west-1.amazonaws.com/public.bitmex.com/data/{}/{}.csv.gz'
    def scrape(year, date,trqu):
        end_date = min(dt(year, 12, 31), dt.today() - timedelta(days=1))
        end_date = dt(2019,11,14)
        while date <= end_date:
            date_str = date.strftime('%Y%m%d')
            print("Processing {} order {}...".format(trqu,date))
            count = 0
            while True:
                r = requests.get(endpoint.format(trqu,date_str))
                if r.status_code == 200:
                    break
                else:
                    count += 1
                    if count == 10:
                        r.raise_for_status()
                    print("Error processing {} - {}, trying again".format(date, r.status_code))
                    time.sleep(10)
            with open(date_str, 'wb') as fp:
                fp.write(r.content)
            with gzip.open(date_str, 'rb') as fp:
                data = fp.read()
            with open(date_str, 'wb') as fp:
                fp.write(data)
            date += timedelta(days=1)
    def merge(year,date,trqu):
        print("Generating CSV for {} order {}".format(trqu,year))
        files = sorted(glob.glob("{}*".format(year)))
        with open("Bitmex_{}_{}-{}-{}.csv".format(trqu,date.day,date.month,year), 'wb') as out:
            first = True
            for f in files:
                with open(f, 'rb') as fp:
                    if first:
                        fp.readline()
                        first = False
                    shutil.copyfileobj(fp, out)
        for f in files:
            os.unlink(f)
    if __name__ == '__main__':
        if len(sys.argv) == 2:
            # if arg is supplied must be in format YYYYMMDD
            # will attempt to remove that file, if exists
            # in case data is incomplete
            year = int(sys.argv[1][:4])
            month = int(sys.argv[1][4:6])
            day = int(sys.argv[1][6:])
            start = dt(year, month, day)
            years = list(range(year, dt.now().year + 1))
            try:
                os.unlink(sys.argv[1])
            except FileNotFoundError:
                pass
        else:
            # 2014-11-12 is the first day of data
            start = dt(2019, month, day)
            years = list(range(2019, dt.now().year + 1))
        starts = [dt(year, month, day) for year in years]
        starts[0] = start

        for trqu in ['trade']:#,'quote']:
            for year, start in zip(years, starts):
                scrape(year, start,trqu)
                merge(year,start,trqu)
    gc.collect()
    
    from datetime import date, datetime, time, timedelta
    chunksize=1000000
#---Chunk Building the midprice data base
   chunksize=1000000
    df_chunk=pd.read_csv('Bitmex_quote_{}-{}-{}.csv'.format(start.day,start.month,start.year),names=['Time','Currency','BestBidSize','BestBidPrice','BestAskPrice','BestAskSize'],chunksize=chunksize)
    chunk_list=[]
    for chunk in df_chunk:
        chunk_filter=chunk[chunk['BestAskPrice']!='askPrice']
        chunk_filter=chunk_filter[chunk_filter['Currency']=='XBTUSD']
        chunk_filter['BestBidPrice']=chunk_filter['BestBidPrice'].astype(float)
        chunk_filter['BestBidSize']=chunk_filter['BestBidSize'].astype(float)
        chunk_filter['BestAskPrice']=chunk_filter['BestAskPrice'].astype(float)
        chunk_filter['BestAskSize']=chunk_filter['BestAskSize'].astype(float)
        chunk_filter['MidPrice']=(chunk_filter['BestBidPrice']+chunk_filter['BestAskPrice'])/2
        chunk_filter['Time'] = chunk_filter['Time'].str.replace('D',' ')
        chunk_filter['Time']=pd.to_datetime(chunk_filter['Time'])
        chunk_filter=chunk_filter.drop(["BestAskPrice","BestAskSize","BestBidPrice","BestBidSize",'Currency'],axis=1)
        chunk_filter=chunk_filter.loc[chunk_filter['MidPrice'].shift(1)!=chunk_filter['MidPrice']]
        chunk_filter=chunk_filter.reset_index().drop('index',axis=1)
        chunk_list.append(chunk_filter)
    df_concat=pd.concat(chunk_list)
    df_concat.to_csv('BTC_Midprice_{}-{}-{}.csv'.format(start.day,start.month,start.year),index = False)
    del df_chunk
    del chunk_list
    del chunk_filter
    del df_concat
    gc.collect()

#---Chunk Building trade tick data base
    df_chunk=pd.read_csv('Bitmex_trade_{}-{}-{}.csv'.format(start.day,start.month,start.year),names=["Time","Currency","Taker Side","Size","Price","TickDirection","TradingID","GrossValue","HomeNotional","ForeignNotional"],chunksize=chunksize)
    chunk_list=[]
    for chunk in df_chunk:
        chunk_filter=chunk[chunk['ForeignNotional']!='foreignNotional']
        chunk_filter=chunk_filter[chunk_filter['Currency']=='XBTUSD']
        chunk_filter=chunk_filter.drop(['Currency','TickDirection','TradingID','GrossValue','HomeNotional','ForeignNotional'],axis=1)
        chunk_filter['Size']=chunk_filter['Size'].astype(float)
        chunk_filter['Price']=chunk_filter['Price'].astype(float)
        chunk_filter['Time'] = chunk_filter['Time'].str.replace('D',' ')
        chunk_filter['Time']=pd.to_datetime(chunk_filter['Time'])
        chunk_filter=chunk_filter.groupby(['Time','Price','Taker Side']).agg({'Size':'sum'})
        chunk_filter=chunk_filter.reset_index()
        chunk_filter=chunk_filter[['Time','Price','Size','Taker Side']]
        chunk_filter=chunk_filter.replace('Sell','SELL')
        chunk_filter=chunk_filter.replace('Buy','BUY')
        chunk_list.append(chunk_filter)
    df_concat2=pd.concat(chunk_list)
    df_concat2.to_csv('BTC_Market_orders_{}-{}-{}.csv'.format(start.day,start.month,start.year),index = False)
    del df_concat2
    del chunk_list
    del chunk_filter
    del df_chunk
    gc.collect()
    
#---Labelling the data
    
main()

Processing trade order 2019-09-29 00:00:00...
Processing trade order 2019-09-30 00:00:00...
Processing trade order 2019-10-01 00:00:00...
Processing trade order 2019-10-02 00:00:00...
Processing trade order 2019-10-03 00:00:00...
Processing trade order 2019-10-04 00:00:00...
Processing trade order 2019-10-05 00:00:00...
Processing trade order 2019-10-06 00:00:00...
Processing trade order 2019-10-07 00:00:00...
Processing trade order 2019-10-08 00:00:00...
Processing trade order 2019-10-09 00:00:00...
Processing trade order 2019-10-10 00:00:00...
Processing trade order 2019-10-11 00:00:00...
Processing trade order 2019-10-12 00:00:00...
Processing trade order 2019-10-13 00:00:00...
Processing trade order 2019-10-14 00:00:00...
Processing trade order 2019-10-15 00:00:00...
Processing trade order 2019-10-16 00:00:00...
Processing trade order 2019-10-17 00:00:00...
Processing trade order 2019-10-18 00:00:00...
Processing trade order 2019-10-19 00:00:00...
Processing trade order 2019-10-20 

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2909: DtypeWarning: Columns (3,4,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [12]:
import pandas as pd
orders = pd.read_csv('BTC_Market_orders_23-7-2019.csv')
orders.Time = pd.to_datetime(orders.Time)
mask = (orders.Time < '2019-08-01')
orders = orders.loc[mask]
orders.to_csv('BTC_Market_orders_23-7-2019_1-8-2019.csv')

# raw quote

In [34]:
from datetime import datetime as dt
def download_raw_quotes(year, month, show_process=False):
#---Importing packagesfrom datetime import datetime as dt
    from datetime import timedelta
    import gzip
    import glob
    import os
    import shutil
    import sys
    import time
    import requests
    import pandas as pd
    import numpy as np
    from math import log,sqrt
    import gc
    
    if 'tmp_bitmex' not in os.listdir():
        os.mkdir('tmp_bitmex')
    
    def get_next_month_dt(date):
        month = date.month
        year = date.year
        next_month = month % 12 + 1
        next_year = year + month // 12
        return dt(next_year, next_month, date.day)
    

#---Importing raw data of trade and quote
    # https://public.bitmex.com/?prefix=data/trade/
    endpoint = 'https://s3-eu-west-1.amazonaws.com/public.bitmex.com/data/{}/{}.csv.gz'
    def scrape(date,trqu):
        end_date = get_next_month_dt(date)
        while date < end_date:
            date_str = date.strftime('%Y%m%d')
            if show_process:
                print("Processing {} order {}...".format(trqu,date))
            count = 0
            while True:
                r = requests.get(endpoint.format(trqu,date_str))
                if r.status_code == 200:
                    break
                else:
                    count += 1
                    if count == 10:
                        r.raise_for_status()
                    print("Error processing {} - {}, trying again".format(date, r.status_code))
                    time.sleep(10)
            with open(date_str, 'wb') as fp:
                fp.write(r.content)
            with gzip.open(date_str, 'rb') as fp:
                data = fp.read()
            with open(date_str, 'wb') as fp:
                fp.write(data)
            date += timedelta(days=1)
    def merge(date,trqu):
        print("{} - Generating CSV for {} order {} {}".format(dt.now(),trqu,date.month,date.year))
        files = sorted(glob.glob("{}*".format(date.strftime('%Y%m'))))
        with open("D://Trading/Data/Crypto/Bitmex/raw_quotes/{}-{}.csv".format(date.month,date.year), 'wb') as out:
            first = True
            for f in files:
                with open(f, 'rb') as fp:
                    if first:
                        fp.readline()
                        first = False
                    shutil.copyfileobj(fp, out)
        for f in files:
            os.unlink(f)
    if __name__ == '__main__':     
        start = dt(year, month, 1)
        trqu = 'quote'
        scrape(start,trqu)
        merge(start,trqu)
        
def extract_quote_symbol(symbol, year, month):
    list_symbols = ['ADAM19', 'BCHM19', 'EOSM19', 'ETHM19', 'ETHUSD', 'LTCM19',
                    'TRXM19', 'XBT7D_D95', 'XBT7D_U105', 'XBTM19', 'XBTU19', 'XBTUSD',
                    'XRPM19']
    from datetime import timedelta
    import gzip
    import glob
    import os
    import shutil
    import sys
    import time
    import requests
    import pandas as pd
    import numpy as np
    from math import log,sqrt
    import gc
    start = dt(year, month, 1)
    
    from datetime import date, datetime, time, timedelta
    chunksize=1000000
#---Chunk Building the midprice data base
    chunksize=1000000
    df_chunk=pd.read_csv('D://Trading/Data/Crypto/Bitmex/raw_quotes/{}-{}.csv'.format(start.month,start.year),
                         names=['Time','Currency','BestBidSize','BestBidPrice','BestAskPrice','BestAskSize'],chunksize=chunksize)
    chunk_list=[]
    for chunk in df_chunk:
        chunk_filter=chunk[chunk['BestAskPrice']!='askPrice']
        chunk_filter=chunk_filter[chunk_filter['Currency']==symbol]
        chunk_filter['BestBidPrice']=chunk_filter['BestBidPrice'].astype(float)
        chunk_filter['BestBidSize']=chunk_filter['BestBidSize'].astype(float)
        chunk_filter['BestAskPrice']=chunk_filter['BestAskPrice'].astype(float)
        chunk_filter['BestAskSize']=chunk_filter['BestAskSize'].astype(float)
        chunk_filter['MidPrice']=(chunk_filter['BestBidPrice']+chunk_filter['BestAskPrice'])/2
        chunk_filter['Time'] = chunk_filter['Time'].str.replace('D',' ')
        chunk_filter['Time']=pd.to_datetime(chunk_filter['Time'])
        chunk_filter=chunk_filter.drop(['Currency'],axis=1)
        chunk_filter=chunk_filter.loc[chunk_filter['MidPrice'].shift(1)!=chunk_filter['MidPrice']]
        chunk_filter=chunk_filter.reset_index().drop('index',axis=1)
        chunk_list.append(chunk_filter)
    df_concat=pd.concat(chunk_list)
    
    path_folder = f'D://Trading/Data/Crypto/Bitmex/quotes/{symbol}/'
    try:
        os.makedirs(path_folder)
    except:
        pass
    filename = '{}.pkl'.format(date.strftime('%Y-%m'))
    df_concat.to_pickle(path_folder_folder+filename, )
    del df_chunk
    del chunk_list
    del chunk_filter
    del df_concat
    gc.collect()
    
    

## raw

In [ ]:
import warnings
warnings.filterwarnings('ignore')
download_raw_quotes(2021, 1)

Processing quote order 2021-01-01 00:00:00...
Processing quote order 2021-01-02 00:00:00...
Processing quote order 2021-01-03 00:00:00...
Processing quote order 2021-01-04 00:00:00...
Processing quote order 2021-01-05 00:00:00...
Processing quote order 2021-01-06 00:00:00...
Processing quote order 2021-01-07 00:00:00...
Processing quote order 2021-01-08 00:00:00...
Processing quote order 2021-01-09 00:00:00...
Processing quote order 2021-01-10 00:00:00...
Processing quote order 2021-01-11 00:00:00...
Processing quote order 2021-01-12 00:00:00...
Processing quote order 2021-01-13 00:00:00...
Processing quote order 2021-01-14 00:00:00...
Processing quote order 2021-01-15 00:00:00...
Processing quote order 2021-01-16 00:00:00...
Processing quote order 2021-01-17 00:00:00...
Processing quote order 2021-01-18 00:00:00...
Processing quote order 2021-01-19 00:00:00...
Processing quote order 2021-01-20 00:00:00...
Processing quote order 2021-01-21 00:00:00...
Processing quote order 2021-01-22 

## loop raw

In [ ]:
import warnings
warnings.filterwarnings('ignore')
for year in [2019,2020,2021]:
    for month in [1,2,3,4,5,6,7,8,9,10,11,12]:
        try:
            print(f'{dt.now()} - dl {year} {month}')
            download_raw_quotes(year, month, show_process=False)
        except:
            print(f'error dl {year} {month}')

## extract symbol

In [23]:
import warnings
warnings.filterwarnings('ignore')
extract_quote_symbol('XBTUSD',2021, 1)

array(['ADAM19', 'BCHM19', 'EOSM19', 'ETHM19', 'ETHUSD', 'LTCM19',
       'TRXM19', 'XBT7D_D95', 'XBT7D_U105', 'XBTM19', 'XBTU19', 'XBTUSD',
       'XRPM19', 'symbol'], dtype=object)

## loop symbol

In [ ]:
import warnings
warnings.filterwarnings('ignore')
list_symbols = ['ADAM19', 'BCHM19', 'EOSM19', 'ETHM19', 'ETHUSD', 'LTCM19', 'TRXM19', 'XBT7D_D95', 'XBT7D_U105', 'XBTM19', 'XBTU19', 'XBTUSD', 'XRPM19']
symbol = 'XBTUSD'
for year in [2019,2020,2021]:
    for month in [1,2,3,4,5,6,7,8,9,10,11,12]:
        try:
            print(f'{dt.now()} - dl {year} {month}')
            extract_quote_symbol(symbol,2021, 1)
        except:
            print(f'error dl {year} {month}')

# raw trades

In [ ]:
from datetime import datetime as dt
def download_raw_trades(year, month):
#---Importing packagesfrom datetime import datetime as dt
    from datetime import timedelta
    import gzip
    import glob
    import os
    import shutil
    import sys
    import time
    import requests
    import pandas as pd
    import numpy as np
    from math import log,sqrt
    import gc
    
    if 'tmp_bitmex' not in os.listdir():
        os.mkdir('tmp_bitmex')
    
    def get_next_month_dt(date):
        month = date.month
        year = date.year
        next_month = month % 12 + 1
        next_year = year + month // 12
        return dt(next_year, next_month, date.day)
    

#---Importing raw data of trade and quote
    # https://public.bitmex.com/?prefix=data/trade/
    endpoint = 'https://s3-eu-west-1.amazonaws.com/public.bitmex.com/data/{}/{}.csv.gz'
    def scrape(date,trqu):
        end_date = get_next_month_dt(date)
        while date < end_date:
            date_str = date.strftime('%Y%m%d')
            print("Processing {} order {}...".format(trqu,date))
            count = 0
            while True:
                r = requests.get(endpoint.format(trqu,date_str))
                if r.status_code == 200:
                    break
                else:
                    count += 1
                    if count == 10:
                        r.raise_for_status()
                    print("Error processing {} - {}, trying again".format(date, r.status_code))
                    time.sleep(10)
            with open(date_str, 'wb') as fp:
                fp.write(r.content)
            with gzip.open(date_str, 'rb') as fp:
                data = fp.read()
            with open(date_str, 'wb') as fp:
                fp.write(data)
            date += timedelta(days=1)
    def merge(date,trqu):
        print("{} - Generating CSV for {} order {} {}".format(dt.now(),trqu,date.month,date.year))
        files = sorted(glob.glob("{}*".format(date.strftime('%Y%m'))))
        with open("D://Trading/Data/Crypto/Bitmex/raw_trades/{}-{}.csv".format(date.month,date.year), 'wb') as out:
            first = True
            for f in files:
                with open(f, 'rb') as fp:
                    if first:
                        fp.readline()
                        first = False
                    shutil.copyfileobj(fp, out)
        for f in files:
            os.unlink(f)
    if __name__ == '__main__':     
        start = dt(year, month, 1)
        trqu = 'trade'
        scrape(start,trqu)
        merge(start,trqu)
        
def extract_trades_symbol(symbol, year, month):
    list_symbols = ['ADAM19', 'BCHM19', 'EOSM19', 'ETHM19', 'ETHUSD', 'LTCM19',
                    'TRXM19', 'XBT7D_D95', 'XBT7D_U105', 'XBTM19', 'XBTU19', 'XBTUSD',
                    'XRPM19']
    from datetime import timedelta
    import gzip
    import glob
    import os
    import shutil
    import sys
    import time
    import requests
    import pandas as pd
    import numpy as np
    from math import log,sqrt
    import gc
    start = dt(year, month, 1)
    
    from datetime import date, datetime, time, timedelta
    chunksize=1000000
    #---Chunk Building trade tick data base
    df_chunk=pd.read_csv('D://Trading/Data/Crypto/Bitmex/raw_trades/{}-{}.csv'.format(start.month,start.year),
                         names=["Time","Currency","Taker Side","Size","Price","TickDirection","TradingID","GrossValue","HomeNotional","ForeignNotional"],chunksize=chunksize)
    chunk_list=[]
    for chunk in df_chunk:
        chunk_filter=chunk[chunk['ForeignNotional']!='foreignNotional']
        chunk_filter=chunk_filter[chunk_filter['Currency']==symbol]
        chunk_filter=chunk_filter.drop(['Currency','TickDirection','TradingID','GrossValue','HomeNotional','ForeignNotional'],axis=1)
        chunk_filter['Size']=chunk_filter['Size'].astype(float)
        chunk_filter['Price']=chunk_filter['Price'].astype(float)
        chunk_filter['Time'] = chunk_filter['Time'].str.replace('D',' ')
        chunk_filter['Time']=pd.to_datetime(chunk_filter['Time'])
        chunk_filter=chunk_filter.groupby(['Time','Price','Taker Side']).agg({'Size':'sum'})
        chunk_filter=chunk_filter.reset_index()
        chunk_filter=chunk_filter[['Time','Price','Size','Taker Side']]
        chunk_filter=chunk_filter.replace('Sell','SELL')
        chunk_filter=chunk_filter.replace('Buy','BUY')
        chunk_list.append(chunk_filter)
    df_concat2=pd.concat(chunk_list)
    
    path_folder = f'D://Trading/Data/Crypto/Bitmex/trades/{symbol}/'
    try:
        os.makedirs(path_folder)
    except:
        pass
    filename = '{}.pkl'.format(date.strftime('%Y-%m'))
    df_concat2.to_pickle(path_folder_folder+filename, )    

    del df_concat2
    del chunk_list
    del chunk_filter
    del df_chunk
    gc.collect()

## raw trade

In [ ]:
import warnings
warnings.filterwarnings('ignore')
download_raw_trades(2021, 1)

## loop raw trade

In [ ]:
import warnings
warnings.filterwarnings('ignore')
for year in [2019,2020,2021]:
    for month in [1,2,3,4,5,6,7,8,9,10,11,12]:
        try:
            print(f'{dt.now()} - dl {year} {month}')
            download_raw_trades(year, month, show_process=False)
        except:
            print(f'error dl {year} {month}')

# exract symbol trade

In [ ]:
import warnings
warnings.filterwarnings('ignore')
extract_trades_symbol('XBTUSD',2021, 1)

## loop extract symbol trade

In [ ]:
import warnings
warnings.filterwarnings('ignore')
list_symbols = ['ADAM19', 'BCHM19', 'EOSM19', 'ETHM19', 'ETHUSD', 'LTCM19', 'TRXM19', 'XBT7D_D95', 'XBT7D_U105', 'XBTM19', 'XBTU19', 'XBTUSD', 'XRPM19']
symbol = 'XBTUSD'
for year in [2019,2020,2021]:
    for month in [1,2,3,4,5,6,7,8,9,10,11,12]:
        try:
            print(f'{dt.now()} - dl {year} {month}')
            extract_trades_symbol(symbol,2021, 1)
        except:
            print(f'error dl {year} {month}')